#Author - Satyam Ramawat (19210520)
##CA684 - Video Memoribility Assignment

**In this work, To Predict video memoriablity the below list of Features has been explored with three Models. Spearman's Correlation Coffiecient Rank Score has been generated in order to understand which feature is best suitable for prediction.**


**A) Features Explored**

1.   *Semantic Feature:* ***Only Captions***
2.   *Semantic Feature:* ***Weighted Captions***
3.   *Video Feature:* **HMP**
4.   *Semantic + Video Feature:* **HMP + Caption**

**Model Used in Each Features**
*   Linear Regression Model
*   Decision Tree Model
*   Random Forest with Estimator 10 & 100

**Final Conclusion Over explored features**

**B) Building Final Machine Learning Model**

Built Final Machine Learning model by using Best Feature after exploring all features.

**C) SAVING RESULTS INTO CSV**

Saving Final Results of ML Model, Short-Term Memoribility and Long-Term Memoribility has been saved in Ground_Truth_Test_Set where CSV name would be **"Ground_Test_Prediction.csv"**.

In [0]:
import pandas
import numpy
import os
import glob
import matplotlib.pyplot as plt

In [0]:
from google.colab import drive
import os
drive.mount('/content/drive/')
os.chdir('/content/drive/My Drive/CA684_Assignment/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


**A) Semantic Feature: Weighted Caption using TFIDF.**

---



**Let's check are we able to reterive files or not**

In [0]:
%ls '/content/drive/My Drive/CA684_Assignment/'

Additional_Resources/  Complete_Sources/  Test-set/
Assignment_Overview/   Dev-set/           Tutorials/


**First training model on ground truth**




In [0]:
!pip install pyprind

In [0]:
GroundTruth_Train = pandas.read_csv('/content/drive/My Drive/CA684_Assignment/Dev-set/Ground-truth/ground-truth.csv')

In [0]:
GroundTruth_Train.head()

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video3.webm,0.924,34,0.846,13
1,video4.webm,0.923,33,0.667,12
2,video6.webm,0.863,33,0.700,10
3,video8.webm,0.922,33,0.818,11
4,video10.webm,0.950,34,0.900,10


**Removing Annotations for Short-Term and Long-Term both, since its not required for the experiment**

In [0]:
GroundTruth_Train = GroundTruth_Train.drop(['nb_short-term_annotations','nb_long-term_annotations'],axis=1)
GroundTruth_Train.head()


,video,short-term_memorability,long-term_memorability
0,video3.webm,0.924,0.846
1,video4.webm,0.923,0.667
2,video6.webm,0.863,0.700
3,video8.webm,0.922,0.818
4,video10.webm,0.950,0.900


Removing .webm from the video column

In [0]:
GroundTruth_Train['video'] = GroundTruth_Train['video'].apply(lambda x : x.split('.')[0])
GroundTruth_Train.head()

,video,short-term_memorability,long-term_memorability
0,video3,0.924,0.846
1,video4,0.923,0.667
2,video6,0.863,0.700
3,video8,0.922,0.818
4,video10,0.950,0.900


**Importing Dev set Captions**

In [0]:
%ls '/content/drive/My Drive/CA684_Assignment/Dev-set/Captions/'

dev-set_video-captions.txt


In [0]:
Captions_Train = pandas.read_csv('/content/drive/My Drive/CA684_Assignment/Dev-set/Captions/dev-set_video-captions.txt',header= None,names=('video','caption'),delimiter='\t')
Captions_Train.head()

,video,caption
0,video3.webm,blonde-woman-is-massaged-tilt-down
1,video4.webm,roulette-table-spinning-with-ball-in-closeup-shot
2,video6.webm,khr-gangsters
3,video8.webm,medical-helicopter-hovers-at-airport
4,video10.webm,couple-relaxing-on-picnic-crane-shot


In [0]:
Captions_Train['video'] = Captions_Train['video'].apply(lambda x : x.split('.')[0])
Captions_Train.head()

,video,caption
0,video3,blonde-woman-is-massaged-tilt-down
1,video4,roulette-table-spinning-with-ball-in-closeup-shot
2,video6,khr-gangsters
3,video8,medical-helicopter-hovers-at-airport
4,video10,couple-relaxing-on-picnic-crane-shot


Binding GroundTruth_Train and Captions_Train

In [0]:
for line in GroundTruth_Train:
    GroundTruth_Train['caption'] = Captions_Train['caption']

In [0]:
GroundTruth_Train.head()

,video,short-term_memorability,long-term_memorability,caption
0,video3,0.924,0.846,blonde-woman-is-massaged-tilt-down
1,video4,0.923,0.667,roulette-table-spinning-with-ball-in-closeup-shot
2,video6,0.863,0.700,khr-gangsters
3,video8,0.922,0.818,medical-helicopter-hovers-at-airport
4,video10,0.950,0.900,couple-relaxing-on-picnic-crane-shot


According to the Media Evaluation paper **Linear Models for Video Memorability Prediction Using Visual and Semantic Features**,  and https://github.com/dazcona/memorability/blob/master/src/notebooks/1.%20EDA%20for%20Captions.ipynb has defined the particular positive words dictionary, thus creating new dictionary in the same way.

For further work, using Natural Language Processing libraries NLTK

In [0]:
import re
import nltk 
nltk.download('stopwords') 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
positive_words_dict = {'women':16,'woman':16,'eating':15,'putting':14,'lying':13,'girl':12,'selfie':11,'relaxing':10,'jellyfish':9,'cat':8,'super':7,'slow':6,'super':5,'american':4,'potrait':3,'pregnant':2,'couple':1}

Below loop will pre-process the data and remove all stopwords, special characters and creating a word bag(Corpus). Further using TFIDFvectorizer in order to calculate word occurance of word bad.

In [0]:
word_bag = []
word_weight = []
for i in range(0, 6000):
    local_weight = 0
    video_caption = re.sub('[^a-zA-Z]', ' ', GroundTruth_Train['caption'][i])
    video_caption = video_caption.lower() 
    video_caption = video_caption.split()
    video_caption = [word for word in video_caption if not word in set(stopwords.words('english'))]
    for word in video_caption:
        if(word in set(positive_words_dict.keys())):
                local_weight = local_weight + positive_words_dict[word]
    word_weight.append(local_weight)
    video_caption = ' '.join(video_caption)
    word_bag.append(video_caption)

## Creating TfidfVectorizer Object
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfV = TfidfVectorizer()

Using Get_score Method in order to generate Spearman's correlations coefficients. Source: https://colab.research.google.com/drive/1QU6mwHNyxi9ycmnbq2A6YtNInvwDizjz#scrollTo=KETcOBrdk0-f

In [0]:
def Get_score(Y_pred,Y_true):
    '''Calculate the Spearmann"s correlation coefficient'''
    Y_pred = numpy.squeeze(Y_pred)
    Y_true = numpy.squeeze(Y_true)
    if Y_pred.shape != Y_true.shape:
        print('Input shapes don\'t match!')
    else:
        if len(Y_pred.shape) == 1:
            Res = pandas.DataFrame({'Y_true':Y_true,'Y_pred':Y_pred})
            score_mat = Res[['Y_true','Y_pred']].corr(method='spearman',min_periods=1)
            print('The Spearman\'s correlation coefficient is: %.3f' % score_mat.iloc[1][0])
        else:
            for ii in range(Y_pred.shape[1]):
                Get_score(Y_pred[:,ii],Y_true[:,ii])

In [0]:
Dataframe_weighted = pandas.DataFrame(numpy.array(word_weight).reshape(6000,1))

**TFIDFVectorizer with three models (LinearRegression, Decision Tree Regression Model and Random Forest Regression model)**

Creating X and Y variables for Train and Test data. 

In [0]:
## Tranforming word bag into tfidfvectorizer
vectorized_Caption = tfidfV.fit_transform(word_bag).toarray()
Vectorized_Dataframe_weighted = numpy.concatenate((vectorized_Caption, Dataframe_weighted), axis=1)

In [0]:
X = Vectorized_Dataframe_weighted
Y = GroundTruth_Train.iloc[:,1:3].values

Spliting data in Train and Test set.

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)

**1.** LinearRegression Model

In [0]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Predicting and finding spearman's correlation cofficient between Actual and X_test

In [0]:
Prediction = lr.predict(X_test)
Get_score(Prediction, y_test)

The Spearman's correlation coefficient is: 0.132
The Spearman's correlation coefficient is: 0.030


**2.** Decision Tree Regression

In [0]:
from sklearn.tree import DecisionTreeRegressor
dtr=DecisionTreeRegressor()
dtr.fit(X_train,y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [0]:
Prediction_2 = dtr.predict(X_test)
Get_score(Prediction_2, y_test)

The Spearman's correlation coefficient is: 0.285
The Spearman's correlation coefficient is: 0.093


**3.** Random Forest Regressor

*Creating model on Default estimator = 10*

In [0]:
from sklearn.ensemble import RandomForestRegressor
rfr_10 = RandomForestRegressor(max_depth= 20,n_estimators=10)
rfr_10.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
Prediction_3 = rfr_10.predict(X_test)
Get_score(Prediction_3, y_test)

The Spearman's correlation coefficient is: 0.294
The Spearman's correlation coefficient is: 0.158


In [0]:
from sklearn.ensemble import RandomForestRegressor
rfr_100 = RandomForestRegressor(max_depth= 20,n_estimators=100)
rfr_100.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
Prediction_4 = rfr_100.predict(X_test)
Get_score(Prediction_4, y_test)

The Spearman's correlation coefficient is: 0.350
The Spearman's correlation coefficient is: 0.193


**B) Semantic Feature: Only Caption using TFIDF.**

Since **Caption with Weight** analysis done above, so not repeating same procedure again instead moving forward with few functional changes

In [0]:
import re
import nltk 
nltk.download('stopwords') 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
word_bag_1 = []
for i in range(0, 6000):
    video_caption = re.sub('[^a-zA-Z]', ' ', GroundTruth_Train['caption'][i])
    video_caption = video_caption.lower() 
    video_caption = video_caption.split()
    video_caption = [word for word in video_caption if not word in set(stopwords.words('english'))]
    video_caption = ' '.join(video_caption)
    word_bag_1.append(video_caption) 

Defining X and Y of dataset

In [0]:
X_1 = tfidfV.fit_transform(word_bag_1).toarray()
Y_1 = GroundTruth_Train.iloc[:, 1:3].values

Spliting data into Test and Train set

In [0]:
from sklearn.model_selection import train_test_split
X1_train, X1_test, y1_train, y1_test = train_test_split(X_1,Y_1, test_size=0.2)

**1.** LinearRegression Model.

In [0]:
from sklearn.linear_model import LinearRegression
lr_1 = LinearRegression()
lr_1.fit(X1_train,y1_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
X.shape

(6000, 5088)

In [0]:
Prediction_5 = lr_1.predict(X1_test)
Get_score(Prediction_5, y1_test)

The Spearman's correlation coefficient is: 0.148
The Spearman's correlation coefficient is: 0.036


**2.** Decision Tree Regressor Model

In [0]:
from sklearn.tree import DecisionTreeRegressor
dtr_1=DecisionTreeRegressor()
dtr_1.fit(X1_train,y1_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [0]:
Prediction_6 = dtr_1.predict(X1_test)
Get_score(Prediction_6, y1_test)

The Spearman's correlation coefficient is: 0.259
The Spearman's correlation coefficient is: 0.127


**3.** Random Forest Regerssor Model 

In [0]:
from sklearn.ensemble import RandomForestRegressor
rfr_10_1 = RandomForestRegressor(max_depth= 20,n_estimators=10)
rfr_10_1.fit(X1_train,y1_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
Prediction_7 = rfr_10_1.predict(X1_test)
Get_score(Prediction_7, y1_test)

The Spearman's correlation coefficient is: 0.289
The Spearman's correlation coefficient is: 0.148


Random Forest Regressor with estimator 100

In [0]:
from sklearn.ensemble import RandomForestRegressor
rfr_100_1 = RandomForestRegressor(max_depth= 20,n_estimators=100)
rfr_100_1.fit(X1_train,y1_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
Prediction_8 = rfr_100_1.predict(X1_test)
Get_score(Prediction_8, y1_test)

The Spearman's correlation coefficient is: 0.325
The Spearman's correlation coefficient is: 0.154


**Conclusion**: Comparison of Spearman's correlation coffiecient scores of Caption and Caption with positive cofficency, **Weighted Captions outperformed** and much better than only Caption.

**C) Video Feature: HMP.**

Using pre-defined HMP read function.

In [0]:
def read_HMP(fname):
    """Scan HMP(Histogram of Motion Patterns) features from file"""
    with open(fname) as f:
        for line in f:
            pairs=line.split()
            HMP_temp = { int(p.split(':')[0]) : float(p.split(':')[1]) for p in pairs}
    # there are 6075 bins, fill zeros
    HMP = numpy.zeros(6075)
    for idx in HMP_temp.keys():
        HMP[idx-1] = HMP_temp[idx]            
    return HMP

In [0]:
%ls '/content/drive/My Drive/CA684_Assignment/Dev-set/'

'Aesthetic Features'/   HMP/                                  ORB/
 C3D/                   HOG/                                  README.txt
 Captions/              InceptionV3/                          Sources/
 ColorHistogram/        LBP/
 Ground-truth/          MemorabilityDataset2018_license.txt


In [0]:
import glob as gb
HMP_list= []
video_list = []
for filename in gb.glob('/content/drive/My Drive/CA684_Assignment/Dev-set/HMP/*.txt'):
    name = ((filename.split('/')[-1]).split('.')[0])
    video_list.append(name)
    HMP_features = read_HMP(filename)
    HMP_list.append(HMP_features)   

In [0]:
HMP_features = pandas.DataFrame(numpy.array(HMP_list).reshape(6000,6075))
HMP_features["video"] = video_list
HMP_features.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,6036,6037,6038,6039,6040,6041,6042,6043,6044,6045,6046,6047,6048,6049,6050,6051,6052,6053,6054,6055,6056,6057,6058,6059,6060,6061,6062,6063,6064,6065,6066,6067,6068,6069,6070,6071,6072,6073,6074,video
0,0.012099,0.001466,0.000017,0.0,0.000683,0.000007,0.0,0.000000,0.000007,0.0,0.0,0.0,0.0,0.000170,0.000002,0.0,0.000002,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000239,0.000324,0.000151,0.000106,0.000050,0.000014,0.000156,0.000123,0.000031,0.000163,0.000177,0.000097,0.000047,0.000021,0.000002,0.000019,0.000021,0.000000,0.000002,0.000009,0.000002,0.000059,0.000024,0.000007,0.000057,0.000052,0.000012,0.000026,0.000007,0.000014,0.000057,0.000028,0.000007,0.000059,0.000097,0.000012,0.000135,0.000111,0.000076,video6634
1,0.078170,0.013597,0.000122,0.0,0.005590,0.000074,0.0,0.000000,0.000026,0.0,0.0,0.0,0.0,0.001137,0.000017,0.0,0.000000,0.000017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000069,0.000091,0.000026,0.000061,0.000022,0.000004,0.000091,0.000048,0.000013,0.000043,0.000095,0.000030,0.000043,0.000013,0.000000,0.000039,0.000026,0.000000,0.000009,0.000013,0.000000,0.000069,0.000026,0.000004,0.000143,0.000139,0.000009,0.000017,0.000043,0.000022,0.000043,0.000026,0.000004,0.000078,0.000074,0.000022,0.000052,0.000082,0.000061,video6633
2,0.112506,0.014053,0.000263,0.0,0.043205,0.000560,0.0,0.000005,0.000364,0.0,0.0,0.0,0.0,0.002472,0.000206,0.0,0.000002,0.000064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000310,0.000629,0.000263,0.000145,0.000061,0.000002,0.000098,0.000248,0.000007,0.000125,0.000354,0.000076,0.000135,0.000049,0.000000,0.000074,0.000103,0.000000,0.000012,0.000039,0.000002,0.000066,0.000012,0.000002,0.000098,0.000221,0.000012,0.000029,0.000076,0.000022,0.000261,0.000069,0.000000,0.000157,0.000558,0.000017,0.000285,0.000504,0.000179,video6639
3,0.066072,0.012682,0.000283,0.0,0.006425,0.000375,0.0,0.000002,0.000174,0.0,0.0,0.0,0.0,0.002373,0.000152,0.0,0.000002,0.000102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000278,0.000744,0.000484,0.000155,0.000077,0.000000,0.000161,0.000563,0.000084,0.000132,0.000375,0.000358,0.000227,0.000075,0.000002,0.000077,0.000135,0.000007,0.000009,0.000011,0.000007,0.000190,0.000049,0.000004,0.000305,0.001002,0.000099,0.000038,0.000106,0.000075,0.000495,0.000210,0.000002,0.000342,0.002027,0.000124,0.000307,0.001239,0.001047,video6635
4,0.030360,0.005834,0.000169,0.0,0.006578,0.000542,0.0,0.000016,0.000373,0.0,0.0,0.0,0.0,0.001041,0.000113,0.0,0.000007,0.000102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000696,0.001019,0.000197,0.000239,0.000111,0.000011,0.000298,0.000389,0.000056,0.000291,0.000395,0.000090,0.000115,0.000052,0.000002,0.000054,0.000070,0.000007,0.000014,0.000041,0.000011,0.000124,0.000045,0.000005,0.000176,0.000181,0.000029,0.000052,0.000090,0.000027,0.000199,0.000061,0.000002,0.000228,0.000253,0.000023,0.000359,0.000375,0.000088,video6637



* Since Ground truth has been already loaded and cleaned into notebook, so avoiding the repetition of same steps of Ground Truth Data. 

* Now concating Ground Truth and HMP into one DataFrame on Video Feature.

In [0]:
HMP_GroundTruth = pandas.merge(GroundTruth_Train,HMP_features, on ='video')
HMP_GroundTruth = HMP_GroundTruth.drop(['caption'],axis=1)
HMP_GroundTruth.head()

,video,short-term_memorability,long-term_memorability,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,6035,6036,6037,6038,6039,6040,6041,6042,6043,6044,6045,6046,6047,6048,6049,6050,6051,6052,6053,6054,6055,6056,6057,6058,6059,6060,6061,6062,6063,6064,6065,6066,6067,6068,6069,6070,6071,6072,6073,6074
0,video3,0.924,0.846,0.125563,0.024036,0.000314,0.0,0.015864,0.000358,0.0,0.000000,0.000086,0.0,0.0,0.0,0.0,0.002795,0.000054,0.0,0.000000,0.000037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000002,0.000173,0.000459,0.000000,0.000148,0.000104,0.000000,0.000121,0.000551,0.000000,0.000114,0.000884,0.000002,0.000116,0.000077,0.000002,0.000027,0.000136,0.000000,0.000000,0.000002,0.000000,0.000091,0.000035,0.000000,0.000163,0.000467,0.000002,0.000010,0.000017,0.000000,0.000393,0.000279,0.000000,0.000289,0.001926,0.000000,0.000086,0.000580,0.000000
1,video4,0.923,0.667,0.007526,0.001421,0.000068,0.0,0.001184,0.000143,0.0,0.000000,0.000079,0.0,0.0,0.0,0.0,0.000246,0.000024,0.0,0.000000,0.000042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000685,0.000308,0.001054,0.000751,0.000176,0.000062,0.000000,0.000123,0.000398,0.000086,0.000246,0.000433,0.000446,0.000143,0.000053,0.000000,0.000053,0.000099,0.000009,0.000004,0.000033,0.000004,0.000051,0.000035,0.000000,0.000062,0.000358,0.000035,0.000024,0.000083,0.000053,0.000244,0.000066,0.000000,0.000081,0.000617,0.000094,0.000220,0.000762,0.001224
2,video6,0.863,0.700,0.109584,0.018978,0.000289,0.0,0.008774,0.000208,0.0,0.000002,0.000088,0.0,0.0,0.0,0.0,0.002046,0.000061,0.0,0.000000,0.000038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000083,0.000057,0.000158,0.000073,0.000021,0.000009,0.000002,0.000019,0.000095,0.000021,0.000019,0.000090,0.000073,0.000050,0.000024,0.000000,0.000012,0.000021,0.000000,0.000000,0.000002,0.000000,0.000017,0.000007,0.000002,0.000066,0.000203,0.000026,0.000002,0.000040,0.000007,0.000054,0.000045,0.000000,0.000028,0.000291,0.000033,0.000052,0.000258,0.000215
3,video8,0.922,0.818,0.120431,0.013561,0.000277,0.0,0.018974,0.000913,0.0,0.000024,0.000713,0.0,0.0,0.0,0.0,0.002496,0.000149,0.0,0.000011,0.000157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000210,0.000434,0.000543,0.000412,0.000412,0.000045,0.000003,0.000144,0.000282,0.000037,0.000197,0.000218,0.000157,0.000237,0.000021,0.000000,0.000040,0.000056,0.000008,0.000005,0.000013,0.000019,0.000168,0.000013,0.000000,0.000133,0.000202,0.000029,0.000029,0.000035,0.000059,0.001110,0.000075,0.000008,0.000333,0.000793,0.000101,0.000588,0.000503,0.000452
4,video10,0.950,0.900,0.005026,0.001356,0.000055,0.0,0.000665,0.000029,0.0,0.000000,0.000024,0.0,0.0,0.0,0.0,0.000147,0.000020,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000160,0.000996,0.001604,0.000103,0.000768,0.000215,0.000009,0.000415,0.000926,0.000020,0.000538,0.001178,0.000050,0.000518,0.000169,0.000007,0.000134,0.000169,0.000007,0.000026,0.000046,0.000007,0.000373,0.000088,0.000000,0.000338,0.000441,0.000029,0.000070,0.000149,0.000009,0.000882,0.000200,0.000009,0.000559,0.001097,0.000018,0.000632,0.001128,0.000064


Creating X and Y variable to pass in Train and Test Dataset.

In [0]:
X_HMP = HMP_GroundTruth.iloc[:,3:6078].values
Y_HMP = HMP_GroundTruth.iloc[:,1:3].values

Splitting Data in Train and Test

In [0]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X_HMP,Y_HMP, test_size=0.2)

**1.** LinearRegression Model.

In [0]:
from sklearn.linear_model import LinearRegression
lr_2 = LinearRegression()
lr_2.fit(X2_train,y2_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
Prediction_9 = lr_2.predict(X2_test)
Get_score(Prediction_9, y2_test)

The Spearman's correlation coefficient is: 0.043
The Spearman's correlation coefficient is: -0.027


**2.** Decision Tree Regressor Model

In [0]:
from sklearn.tree import DecisionTreeRegressor
dtr_2=DecisionTreeRegressor()
dtr_2.fit(X2_train,y2_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [0]:
Prediction_10 = dtr_2.predict(X2_test)
Get_score(Prediction_10, y2_test)

The Spearman's correlation coefficient is: 0.087
The Spearman's correlation coefficient is: 0.043


**3.** Random Forest Regerssor Model 

In [0]:
from sklearn.ensemble import RandomForestRegressor
rfr_10_2 = RandomForestRegressor(max_depth= 20,n_estimators=10)
rfr_10_2.fit(X2_train,y2_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
Prediction_11 = rfr_10_2.predict(X2_test)
Get_score(Prediction_11, y2_test)

The Spearman's correlation coefficient is: 0.224
The Spearman's correlation coefficient is: 0.056


With estimator = 100

In [0]:
from sklearn.ensemble import RandomForestRegressor
rfr_100_2 = RandomForestRegressor(max_depth= 20,n_estimators=100)
rfr_100_2.fit(X2_train,y2_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
Prediction_12 = rfr_100_2.predict(X2_test)
Get_score(Prediction_12, y2_test)

The Spearman's correlation coefficient is: 0.321
The Spearman's correlation coefficient is: 0.112


**D) Semantic + Video feature: HMP + Captions.**

Since all pre-processing has been done for both HMP and Caption feature, thus use the variables for prediction.

Merging HMP[3-6078] and Caption into one frame.

In [0]:
HMP_Caption = numpy.concatenate((X_1, X_HMP), axis=1)

Defining X and Y

In [0]:
X_HMP_Caption = HMP_Caption
Y_HMP_Caption= GroundTruth_Train.iloc[:, 1:3].values

Spliting Data into train and test

In [0]:
X3_train, X3_test, y3_train, y3_test = train_test_split(X_HMP_Caption,Y_HMP_Caption, test_size=0.3)

**1.** LinearRegression Model.


In [0]:
from sklearn.linear_model import LinearRegression
lr_3 = LinearRegression()
lr_3.fit(X3_train,y3_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
Prediction_13 = lr_3.predict(X3_test)
Get_score(Prediction_13, y3_test)

The Spearman's correlation coefficient is: -0.008
The Spearman's correlation coefficient is: 0.038


**2.** Decision Tree Regressor Model

In [0]:
from sklearn.tree import DecisionTreeRegressor
dtr_3=DecisionTreeRegressor()
dtr_3.fit(X3_train,y3_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [0]:
Prediction_14 = dtr_3.predict(X3_test)
Get_score(Prediction_14, y3_test)

The Spearman's correlation coefficient is: 0.093
The Spearman's correlation coefficient is: 0.047


**3.** Random Forest Regressor Model

Estimator = 10

In [0]:
from sklearn.ensemble import RandomForestRegressor
rfr_10_3 = RandomForestRegressor(max_depth= 20,n_estimators=10)
rfr_10_3.fit(X3_train,y3_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
Prediction_15 = rfr_10_3.predict(X3_test)
Get_score(Prediction_15, y3_test)

The Spearman's correlation coefficient is: 0.224
The Spearman's correlation coefficient is: 0.035


Estimator = 100

In [0]:
from sklearn.ensemble import RandomForestRegressor
rfr_100_3 = RandomForestRegressor(max_depth= 20,n_estimators=100)
rfr_100_3.fit(X3_train,y3_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
Prediction_11 = rfr_100_3.predict(X3_test)
Get_score(Prediction_11, y3_test)

The Spearman's correlation coefficient is: 0.318
The Spearman's correlation coefficient is: 0.138


**Conclusion**

**Overall, Spearman's Correlation Score has been calculated for Caption, Caption with Weight, HMP and HMP+Caption. Where Captions with outperformed from all the models therefore, proceeding with Weighted Caption models for final Machine Learning model for prediction.**

### **B) Final Step: Building Machine Learning Model on Semantic Feature: Weighted Caption.**

In [0]:
GroundTruth_Train_Final = pandas.read_csv('/content/drive/My Drive/CA684_Assignment/Dev-set/Ground-truth/ground-truth.csv')

In [0]:
GroundTruth_Train_Final = GroundTruth_Train_Final.drop(['nb_short-term_annotations','nb_long-term_annotations'],axis=1)
GroundTruth_Train_Final.head()

,video,short-term_memorability,long-term_memorability
0,video3.webm,0.924,0.846
1,video4.webm,0.923,0.667
2,video6.webm,0.863,0.700
3,video8.webm,0.922,0.818
4,video10.webm,0.950,0.900


In [0]:
%ls '/content/drive/My Drive/CA684_Assignment/Test-set/Ground-truth_test/'

ground_truth_template.csv


In [0]:
GroundTruth_Test = pandas.read_csv('/content/drive/My Drive/CA684_Assignment/Test-set/Ground-truth_test/ground_truth_template.csv')
GroundTruth_Test.head()

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,7494,NaN,33,NaN,12
1,7495,NaN,34,NaN,10
2,7496,NaN,32,NaN,13
3,7497,NaN,33,NaN,10
4,7498,NaN,33,NaN,10


Importing caption for train and test both.

In [0]:
Caption_Train = pandas.read_csv('/content/drive/My Drive/CA684_Assignment/Dev-set/Captions/dev-set_video-captions.txt',names=('video','caption'),delimiter='\t',header= None)
Caption_Train.head()

,video,caption
0,video3.webm,blonde-woman-is-massaged-tilt-down
1,video4.webm,roulette-table-spinning-with-ball-in-closeup-shot
2,video6.webm,khr-gangsters
3,video8.webm,medical-helicopter-hovers-at-airport
4,video10.webm,couple-relaxing-on-picnic-crane-shot


In [0]:
Caption_Test = pandas.read_csv('/content/drive/My Drive/CA684_Assignment/Test-set/Captions_test/test-set-1_video-captions.txt',names=('video','caption'),delimiter='\t',header= None)
Caption_Test.head()

,video,caption
0,video7494.webm,green-jeep-struggling-to-drive-over-huge-rocks
1,video7495.webm,hiking-woman-tourist-is-walking-forward-in-mou...
2,video7496.webm,close-up-of-african-american-doctors-hands-usi...
3,video7497.webm,slow-motion-of-a-man-using-treadmill-in-the-gy...
4,video7498.webm,slow-motion-of-photographer-in-national-park


Binding both Train and Test Captions

In [0]:
Captions_Train_Test = pandas.concat([Caption_Train,Caption_Test],ignore_index=True)
Captions_Train_Test.head()

,video,caption
0,video3.webm,blonde-woman-is-massaged-tilt-down
1,video4.webm,roulette-table-spinning-with-ball-in-closeup-shot
2,video6.webm,khr-gangsters
3,video8.webm,medical-helicopter-hovers-at-airport
4,video10.webm,couple-relaxing-on-picnic-crane-shot


In [0]:
import re
import nltk 
nltk.download('stopwords') 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

word_bag = []
word_weight = []
for i in range(0, 6000):
    local_weight = 0
    video_caption = re.sub('[^a-zA-Z]', ' ', Captions_Train_Test['caption'][i])
    video_caption = video_caption.lower() 
    video_caption = video_caption.split()
    video_caption = [word for word in video_caption if not word in set(stopwords.words('english'))]
    for word in video_caption:
        if(word in set(positive_words_dict.keys())):
                local_weight = local_weight + positive_words_dict[word]
    word_weight.append(local_weight)
    video_caption = ' '.join(video_caption)
    word_bag.append(video_caption)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
word_weight_dataframe = pandas.DataFrame(numpy.array(word_weight).reshape(6000,1))

In [0]:
## Creating TfidfVectorizer Object
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfV = TfidfVectorizer()


In [0]:
Captions = tfidfV.fit_transform(word_bag).toarray()
Bind_Caption_Word = numpy.concatenate((Captions,word_weight_dataframe), axis=1)

In [0]:
print(Captions.shape)
print(Bind_Caption_Word.shape)

(6000, 5087)
(6000, 5088)


Now Merged Captions and corpus array will be splited manually into Train and Test.

In [0]:
X4_train = Bind_Caption_Word[:4000]
X4_test = Bind_Caption_Word[4000:]
print(X4_train.shape)
print(X4_test.shape)

(4000, 5088)
(2000, 5088)


In [0]:
Y4_train = GroundTruth_Train_Final.iloc[:4000, 1:3].values
Y4_train.shape

(4000, 2)

In [0]:
Y4_test = GroundTruth_Train_Final.iloc[4000:, 1:3].values
Y4_test.shape

(2000, 2)

***Using Random Forest Regressor with estimator = 100 since it outperformed from Linear Regressor and Decision Tree Regressor and obtain good Spearman's correlation cofficient score.***

In [0]:
from sklearn.ensemble import RandomForestRegressor
rfr_100_4 = RandomForestRegressor(max_depth= 20,n_estimators=100)
rfr_100_4.fit(X4_train,Y4_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
Prediction_12 = rfr_100_4.predict(X4_test)
Get_score(Prediction_12, Y4_test)

The Spearman's correlation coefficient is: 0.318
The Spearman's correlation coefficient is: 0.173


In [0]:
print(Prediction_12.shape)
print(type(Prediction_12))

(2000, 2)
<class 'numpy.ndarray'>


### **C) Saving Results**

In [0]:
Final = numpy.asarray(Prediction_12)
numpy.savetxt("/content/drive/My Drive/Prediction_Video_Memorability.csv", Final, delimiter=",")

In [0]:
Prediction_Video_Memorability = pandas.read_csv('/content/drive/My Drive/Prediction_Video_Memorability.csv', names=['short-term_memorability','long-term_memorability'])
Prediction_Video_Memorability.head()

,short-term_memorability,long-term_memorability
0,0.847892,0.765770
1,0.906731,0.855989
2,0.896926,0.826620
3,0.856703,0.777345
4,0.857068,0.780128


In [0]:
GroundTruth_Test_new = GroundTruth_Test.drop(['short-term_memorability','long-term_memorability'],axis=1)

In [0]:
FinalGroundTruth_Test = pandas.concat([GroundTruth_Test_new,Prediction_Video_Memorability],ignore_index=False,axis=1)
FinalGroundTruth_Test.head()


,video,nb_short-term_annotations,nb_long-term_annotations,short-term_memorability,long-term_memorability
0,7494,33,12,0.847892,0.765770
1,7495,34,10,0.906731,0.855989
2,7496,32,13,0.896926,0.826620
3,7497,33,10,0.856703,0.777345
4,7498,33,10,0.857068,0.780128


In [0]:
FinalGroundTruth_Test.shape

(2000, 5)

In [0]:
export_csv = FinalGroundTruth_Test.to_csv (r'/content/drive/My Drive/Ground_Test_Prediction.csv', index = None, header=True)

**----------------------------------------------------------------------------------END OF WORK-----------------------------------------------------------------**